In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2 
import numpy as np
import h5py
from tensorflow.keras.models import load_model

In [2]:
cnn_model = load_model('ASL_MNIST_CNN_20.h5')

In [30]:
lookUpTable1 = np.empty(256, np.uint8)
for i in range(256):
    lookUpTable1[i] = np.clip(pow(i / 255.0, 0.5) * 255.0, 0, 255)
        
lookUpTable2 = np.empty(256, np.uint8)
for i in range(256):
    lookUpTable2[i] = np.clip(pow(i / 255.0, 2) * 255.0, 0, 255)

In [31]:
def transform(img, table):
    w, h = img.shape
    tile = img.reshape(w*h)
    trans = table[tile]
    reshaping = trans.reshape(w, h)
    return reshaping

In [33]:
### Setting the number of the camera
cap = cv2.VideoCapture(0)
### Setting the resolusion of the camera (720p)
img_width = 1280
img_height = 720
cap.set(cv2.CAP_PROP_FRAME_WIDTH, img_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, img_height)

cut_point1 = int((img_width - img_height ) / 2) 
cut_point2 = int((img_width + img_height ) / 2)

org1 = int(cut_point1 / 2)
org2 = int(img_height / 2)

org = (int(cut_point1 / 2), int(img_height / 2))
fontFace = cv2.FONT_HERSHEY_COMPLEX
fontScale = 5
fontcolor = (0, 0, 255) # BGR
thickness = 5 
lineType = 4
n = 0


while 1:
    ret, frame = cap.read()
    
    frame_o = frame
    
    
    ####################################################
    # Frame processing
    
    crop = frame_o[:, cut_point1:cut_point2]
    
    grayscale = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    
    transform_1 = transform(grayscale, lookUpTable1)
        
    gaussian = cv2.GaussianBlur(transform_1, (31,31),0)
    
    ret2, thresholding = cv2.threshold(gaussian ,0 ,255 , cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    transform_2 = transform(gaussian, lookUpTable2)


    mask = thresholding==255
    masked = 255 - (255 - transform_2) * mask
    
    
    resized = cv2.resize(masked,(28,28))
    ####################################################
    
    ####################################################
    input_ = resized.reshape(-1,28,28,1)
    y_pred = cnn_model.predict_proba(input_)
    #print(y_pred)
    num = np.argmax(y_pred)   
    
    if num <= 8:
        letter = chr(num + 65)
    else:
        letter = chr(num + 66)

    cv2.putText(frame_o, letter, org, fontFace, fontScale, fontcolor, thickness, lineType)
    cv2.rectangle(frame_o, (cut_point1 ,0), (cut_point2, img_height-1), (0, 255,0),5)
    cv2.imshow('video',frame_o)
    cv2.imshow('v2',input_[0])
    
    key = cv2.waitKey(1)
    ### Press Esc to quit detecting
    if key == 27:
        break
    
    elif key == ord('s'):
        cv2.imwrite('/home/ghw/Desktop/project_imgs/Y/test_%s.png' %n ,frame)
        print('save done!'+str(n))
        n = n+1
        
        

cap.release()
cv2.destroyAllWindows()        

In [18]:
test = cv2.imread('/home/ghw/Desktop/test.png')


crop = test[:, cut_point1:cut_point2]
cv2.imwrite('/home/ghw/Desktop/testimgs/crop.png' ,crop)

grayscale = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
cv2.imwrite('/home/ghw/Desktop/testimgs/grayscale.png' ,grayscale)

transform_1 = cv2.transform(grayscale, lookUpTable1)
cv2.imwrite('/home/ghw/Desktop/testimgs/transform_1.png' ,transform_1)

gaussian = cv2.GaussianBlur(transform_1, (11,11),0)
cv2.imwrite('/home/ghw/Desktop/testimgs/gaussian.png' ,gaussian)

ret2, thresholding = cv2.threshold(gaussian ,0 ,255 , cv2.THRESH_BINARY + cv2.THRESH_OTSU)
cv2.imwrite('/home/ghw/Desktop/testimgs/thresholding.png' ,thresholding)

transform_2 = cv2.transform(gaussian, lookUpTable2)
cv2.imwrite('/home/ghw/Desktop/testimgs/transform_2.png' ,transform_2)

mask = thresholding==255
masked = 255 - (255 - transform_2) * mask
cv2.imwrite('/home/ghw/Desktop/testimgs/masked.png' ,masked)

resized = cv2.resize(masked,(28,28))
cv2.imwrite('/home/ghw/Desktop/testimgs/resized.png' ,resized)

input_ = resized.reshape(-1,28,28,1)
y_pred = cnn_model.predict(input_)
num = np.argmax(y_pred)   

if num <= 8:
    letter = chr(num + 65)
else:
    letter = chr(num + 66)
cv2.putText(test, letter, org, fontFace, fontScale, fontcolor, thickness, lineType)
cv2.rectangle(test, (cut_point1 ,0), (cut_point2, img_height-1), (0, 255,0),5)
cv2.imwrite('/home/ghw/Desktop/testimgs/pred.png' ,test)






TypeError: 'NoneType' object is not subscriptable